In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Step 1: Set Paths
DATASET_DIR = "/path/to/leapGestRecog"  # update this to your local path

# Step 2: Load Images
def load_images(data_dir, img_size=(64, 64), limit_per_class=None):
    images = []
    labels = []
    label_map = {}

    folders = sorted(os.listdir(data_dir))
    for label_id, folder in enumerate(folders):
        label_map[label_id] = folder
        folder_path = os.path.join(data_dir, folder)
        image_files = os.listdir(folder_path)
        count = 0
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(label_id)
                count += 1
                if limit_per_class and count >= limit_per_class:
                    break
    return np.array(images), np.array(labels), label_map

X, y, label_map = load_images(os.path.join(DATASET_DIR, "00"), img_size=(64, 64))  # Load person 00
X = X.reshape(-1, 64, 64, 1) / 255.0
y = to_categorical(y, num_classes=10)

# Step 3: Train-test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

# Step 6: Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Step 7: Predict Gesture
def predict_gesture(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64, 64))
    img = img.reshape(1, 64, 64, 1) / 255.0
    pred = model.predict(img)
    class_idx = np.argmax(pred)
    gesture_label = label_map[class_idx]
    print(f"Predicted Gesture: {gesture_label}")
    plt.imshow(img.reshape(64,64), cmap='gray')
    plt.title(f"Gesture: {gesture_label}")
    plt.axis('off')
    plt.show()

# Example
# predict_gesture("/path/to/leapGestRecog/00/01/01_1.jpg")
